In [ ]:
%reload_ext autoreload
%autoreload 2

In [4]:
a = np.random.random((20))

In [5]:
import cv2 
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, ConcatDataset
from os import path 
import os
from utilities.pipeline import *
from utilities.postprocessing.utils import KrockPatchExtractStrategy
import os 
from utilities.postprocessing.handlers import *
import zipfile
import glob 
# import tqdm
from tqdm import tqdm_notebook as bar
import pypeln.thread as th
from pydrive.auth import GoogleAuth

drive_ids = {
    'test': '1tN6jcMHiwLfDWogJ4YvbistPNlodikPI'
}

download_dir= '/home/francesco/Desktop/'

class TraversabilityDataset(Dataset):
    def __init__(self, df, images_dir, time_window):
        self.df, self.images_dir, self.time_window = df, images_dir, time_window
        
        self.df = add_advancement(df, time_window)

    def __getitem__(self, idx):
        img_path = self.df['images'][idx]
        im = cv2.imread(self.images_dir + '/' + img_path)
        return im
    
    def __len__(self):
        
        return len(self.df)
    
    @classmethod
    def from_remote(cls, url, which='test', where=download_dir, *args, **kwargs):
        zip_path = where + '/' + which + '.zip'
        print('[INFO] Downloading using curl in {}...'.format(where), end='')
#         old school never dies
        os.system('curl -L -o {} "https://drive.google.com/uc?export=download&id={}"'.format(zip_path, drive_ids[which]))
        print('done!')
        return cls.from_zip(zip_path, out_dir=where, *args, **kwargs)
    
    @classmethod
    def from_zip(cls, zip_path, out_dir=download_dir, *args, **kwargs):
        zip_ref = zipfile.ZipFile(zip_path, 'r')
        extracted = zip_ref.namelist()
        root = path.join(out_dir, extracted[0])
        
        should_unzip = not path.isdir(root)
        if should_unzip:
            print('[INFO] Extracting Zip file to {}...'.format(root), end='')
            zip_ref.extractall(out_dir)
            zip_ref.close()
        else:
            print('[INFO] Loading from {}...'.format(root), end='')
        print('done!')
        return cls.from_root(root=root, *args, **kwargs)
    
    @classmethod
    def from_meta(cls, meta, root, max_advancement, maps_dir, out_dir=None, time_window=None, *args, **kwargs):
        ds = []
        if out_dir is None: out_dir = root
            
        meta_out_dir = "{}/csvs/".format(out_dir)
        patches_out_dir = "{}/patches/{}/".format(out_dir, max_advancement)
        os.makedirs(meta_out_dir, exist_ok=True)
        os.makedirs(patches_out_dir, exist_ok=True)

        should_extract_patches = len(glob.glob(patches_out_dir + '/*.png')) <= 0
        if should_extract_patches: 
            print('[INFO] extracting patches to {}...'.format(patches_out_dir), end='')
            patches_extractor = MultiThreadWrapper(16, Compose([
                ReadDataframeFilenameAndHm(meta_out_dir, maps_dir),
#                 AddAdvancement(time_window),
                ExtractPatches(patch_extract_stategy=KrockPatchExtractStrategy(max_advancement=max_advancement)),
                StorePatches(patches_out_dir,meta_out_dir)
            ]))

            out = patches_extractor(meta.iterrows())
            print('done!')
        
        print('[INFO] Creating datasets...', end='')
        
        pbar = bar(meta.iterrows())
#         create all datasets       
        for idx, row in pbar:       
            filename, map = row['filename'], row['map']
            pbar.set_description('[INFO] {}'.format(filename))
            try:
                df, hm = open_df_and_hm_from_meta_row(row, meta_out_dir, maps_dir)
                ds.append(cls(df, patches_out_dir,time_window))
            except FileNotFoundError:
                continue
                
        print('done!')
        return ConcatDataset(ds)
    
    @classmethod
    def from_root(cls, root, max_advancement, maps_dir=None, out_dir=None, *args, **kwargs):
        maps_dir = root + '/maps/' if maps_dir is None else maps_dir
        out_dir = root if out_dir is None else out_dir
        
        meta = pd.read_csv(root + '/meta.csv')
        
        return cls.from_meta(meta, root, max_advancement, maps_dir, out_dir, *args, **kwargs)

In [7]:
ds = TraversabilityDataset.from_root('/media/francesco/saetta/krock-dataset/test',  
                                  max_advancement=0.71,
                               time_window=100)

print(ds[0])


0it [00:00, ?it/s]

[INFO] extracting patches to /media/francesco/saetta/krock-dataset/test/patches/0.66/...

FileNotFoundError: 

Original Traceback (most recent call last):
  File "/home/francesco/.local/lib/python3.6/site-packages/pypeln/thread.py", line 164, in wrapper
    return f(*args, **kwargs)
  File "/home/francesco/.local/lib/python3.6/site-packages/pypeln/thread.py", line 214, in f_task
    y = f(x, *args)
  File "/home/francesco/Documents/Master-Thesis/core/utilities/pipeline/pipeline.py", line 15, in __call__
    res = self.handlers[0](*args, **kwargs)
  File "/home/francesco/Documents/Master-Thesis/core/utilities/postprocessing/handlers/handlers.py", line 33, in __call__
    df, hm = open_df_and_hm_from_meta_row(row, self.base_dir, self.hm_dir)
  File "/home/francesco/Documents/Master-Thesis/core/utilities/postprocessing/handlers/functional.py", line 171, in open_df_and_hm_from_meta_row
    df = pd.read_csv(base_dir + '/' + filename + '.csv')
  File "/home/francesco/.local/lib/python3.6/site-packages/pandas/io/parsers.py", line 697, in parser_f
    return _read(filepath_or_buffe

In [22]:
ds = TraversabilityDataset.from_remote('asd',  
                                  max_advancement=0.71,
                                     maps_dir='./maps/new-train/',
                                     time_window=100)

print(ds[0])

[INFO] Downloading using curl in /home/francesco/Desktop/...done!
[INFO] Extracting Zip file to /home/francesco/Desktop/slope/...done!
[INFO] Creating datasets...


done!
[[[ 72  72  72]
  [ 73  73  73]
  [ 73  73  73]
  ...
  [114 114 114]
  [115 115 115]
  [116 116 116]]

 [[ 72  72  72]
  [ 73  73  73]
  [ 73  73  73]
  ...
  [114 114 114]
  [115 115 115]
  [116 116 116]]

 [[ 72  72  72]
  [ 73  73  73]
  [ 73  73  73]
  ...
  [114 114 114]
  [115 115 115]
  [116 116 116]]

 ...

 [[ 75  75  75]
  [ 76  76  76]
  [ 76  76  76]
  ...
  [119 119 119]
  [119 119 119]
  [120 120 120]]

 [[ 75  75  75]
  [ 76  76  76]
  [ 76  76  76]
  ...
  [119 119 119]
  [119 119 119]
  [120 120 120]]

 [[ 75  75  75]
  [ 76  76  76]
  [ 76  76  76]
  ...
  [118 118 118]
  [119 119 119]
  [120 120 120]]]


In [ ]:
ds = TraversabilityDataset.from_zip(
                                    zip_path='/media/francesco/saetta/krock-dataset/val.zip',
                                    out_dir='/media/francesco/saetta/zip/',
                                     max_advancement=0.66,
                                     maps_dir='./maps/val/',
                                     time_window=100)

print(ds[0])

In [ ]:
meta = pd.read_csv('/home/francesco/Desktop/test/meta.csv')
ds = TraversabilityDataset.from_meta(meta,
                                     root='/media/francesco/saetta/krock-dataset/train/',
                                     out_dir='/home/francesco/Desktop/test/',
                                     max_advancement=0.66,
                                     maps_dir='./maps/train/',
                                     time_window=100)

print(ds[0])

In [ ]:
df = pd.read_csv('/media/francesco/saetta/krock-dataset/train/csvs/slope_rocks1-7.0-26.csv')
meta = pd.read_csv('/home/francesco/Desktop/test/meta.csv')

hm = cv2.imread('./maps/new-train/slope_rocks1.png')
hm = cv2.cvtColor(hm, cv2.COLOR_BGR2GRAY)

ds = TraversabilityDataset.from_df(df, hm, 0.66,
                                               out_dir='/home/francesco/Desktop/test/')

ds[0]

In [ ]:
from utilities.postprocessing.handlers import *

In [ ]:
from utilities.pipeline import *

In [7]:
a = np.random.random((20))
b = np.random.random((80))

tot = 100


array([0.61069966, 0.75589338, 0.18784141, 0.9053972 , 0.40566203,
       0.59764261, 0.04879094, 0.40811138, 0.48616502, 0.25838815,
       0.24416049, 0.4908535 , 0.90192317, 0.05312703, 0.75706453,
       0.60262945, 0.1131344 , 0.25877077, 0.85337593, 0.27058206])